# **Введение**

---
Это Pet-project по A/B тестированию на основе данных о поведении покупателей на сайте.

**Тестируемое изменение:** Фон на сайте сменили с чёрного цвета на белый

**Какие метрики мы будем отслеживать:**

*   Целевая метрика: Конверсия в целевое действие
*   Среднее количество просмотренных страниц
*   Среднее время проведенное на сайте

**H0:** Конверсия в тестовой группе не отличается от конверсии в контрольной группе

**Н1:** Конверсия в тестовой группе отличается от конверсии в контрольной группе

**p-уровень значимости:** 0.05



---






# **Загружаем дата сет и проверим:**

1.   Размер датасета
2.   Наличие пропусков, ошибок в данных
3.   Корректность сплита, тест на SRE






In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("ab_testing.csv")
df.columns = [c.strip() for c in df.columns]
df["converted"] = (df["Conversion"].astype(str).str.strip().str.lower() == "yes").astype(int)
df["is_B"] = (df["Group"].astype(str).str.strip().str.upper() == "B").astype(int)
print(df.shape)
print(df.dtypes)
print(df.isna().sum())
print(df["Group"].value_counts())
print(df["Conversion"].value_counts())


FileNotFoundError: [Errno 2] No such file or directory: 'ab_testing.csv'

In [ ]:
n_rows = df.shape[0]
n_users = df["User ID"].nunique()
dup_users = n_rows - n_users
print("rows:", n_rows)
print("unique users:", n_users)
print("duplicate user rows:", dup_users)
print(df[["Page Views", "Time Spent"]].describe())


rows: 5000
unique users: 5000
duplicate user rows: 0
        Page Views   Time Spent
count  5000.000000  5000.000000
mean      7.537600   242.512600
std       4.022465   118.254153
min       1.000000    40.000000
25%       4.000000   137.000000
50%       8.000000   243.000000
75%      11.000000   346.000000
max      14.000000   449.000000


**В датасете 5000 строк и 7 колонок, имеются все необходимые значения без пропусков. User ID уникален (5000 уникальных из 5000 строк)**

**По распределению трафика группы почти равны: A = 2519, B = 2481, корректность сплита соответствует описанию дизайна теста**

---




In [ ]:
from scipy.stats import chisquare
counts = df["Group"].value_counts()
n_a = int(counts.get("A", 0))
n_b = int(counts.get("B", 0))
obs = np.array([n_a, n_b])
exp = np.array([0.5 * obs.sum(), 0.5 * obs.sum()])
chi2, p_srm = chisquare(f_obs=obs, f_exp=exp)
print("n_A:", n_a, "n_B:", n_b)
print("SRM chi2:", chi2, "p-value:", p_srm)

n_A: 2519 n_B: 2481
SRM chi2: 0.2888 p-value: 0.5909905465803058


Проверка на SRM дала хороший результат, значит пользователи корректно распределены по группам



In [ ]:
from scipy.stats import chi2_contingency

ct_device = pd.crosstab(df["Group"], df["Device"])
ct_location = pd.crosstab(df["Group"], df["Location"])

p_device = chi2_contingency(ct_device)[1]
p_location = chi2_contingency(ct_location)[1]

print(ct_device)
print("device balance p-value:", p_device)

print(ct_location)
print("location balance p-value:", p_location)


Device  Desktop  Mobile
Group                  
A          1244    1275
B          1294    1187
device balance p-value: 0.05338692515743378
Location  England  Northern Ireland  Scotland  Wales
Group                                               
A             606               614       670    629
B             640               628       611    602
location balance p-value: 0.2501782724258139


Группы однородны по своим девайсам и географии, что подтверждает полученный p-value (<0.05)


---






## **Подсчет метрик**
Теперь мы можем перейти к рассчету выбранных метрик и проверить нулевую гипотезу

In [ ]:
summary = df.groupby("Group")["converted"].agg(["count", "sum", "mean"])
summary = summary.rename(columns={"count": "n", "sum": "conversions", "mean": "conversion_rate"})
print(summary)

from statsmodels.stats.proportion import proportions_ztest, proportion_confint

conv_a = int(summary.loc["A", "conversions"])
conv_b = int(summary.loc["B", "conversions"])
n_a = int(summary.loc["A", "n"])
n_b = int(summary.loc["B", "n"])

count = np.array([conv_a, conv_b])
nobs = np.array([n_a, n_b])

z_stat, p_value = proportions_ztest(count=count, nobs=nobs, alternative="two-sided")

print("z:", z_stat)
print("p-value:", p_value)


          n  conversions  conversion_rate
Group                                    
A      2519          136         0.053990
B      2481          349         0.140669
z: -10.354488520266216
p-value: 3.9931511894201938e-25


Количество пользователей, совершивших целевое действие:

A: 136 из 2519, конверсия ≈ **0.0540**

B: 349 из 2481, конверсия ≈ **0.1407**

p-value < 0.05

Тест дал показал, что в тестовой группе конверсия в целевое действие статистически значимо выше, чем в контрольной. Uplift составляет 8,6 п.п.


---



In [ ]:
from scipy.stats import ttest_ind, mannwhitneyu

a_pv = df[df["Group"] == "A"]["Page Views"]
b_pv = df[df["Group"] == "B"]["Page Views"]
a_ts = df[df["Group"] == "A"]["Time Spent"]
b_ts = df[df["Group"] == "B"]["Time Spent"]

print(df.groupby("Group")[["Page Views", "Time Spent"]].agg(["mean", "median"]))

t_pv = ttest_ind(a_pv, b_pv, equal_var=False).pvalue
t_ts = ttest_ind(a_ts, b_ts, equal_var=False).pvalue

print("t-test p Page Views:", t_pv)
print("t-test p Time Spent:", t_ts)

NameError: name 'df' is not defined



Тест показал, что в тестовой группе среднее количество просмотренных страниц и проведенное время статистически значимо не отличается от контрольной. Можно сделать вывод, что изменение фона не привело к увеличению длинны сессии

---



# **Вывод**





---
На основе полученных результатов, можно сделать вывод о том, что поменять фон сайта было хорошей идеей, так как конверсия увеличилась из-за этого, мы добились успеха! 🔥
